In [9]:
import pandas as pd
import numpy as np

data_raw = pd.read_csv('../raw/지역사회건강조사 1차 전처리_2017.csv')

In [10]:
df = data_raw[['만나이','성별','사회환경_동네 자연환경','자치구']]

In [11]:
df_ne1=df[df['사회환경_동네 자연환경'] != 9]
df_ne2=df_ne1[df_ne1['사회환경_동네 자연환경'] != 7]
df_ne2

,만나이,성별,사회환경_동네 자연환경,자치구
0,82,2,1,강남구
1,51,2,1,강남구
2,26,1,2,강남구
3,24,1,1,강남구
4,27,2,1,강남구
...,...,...,...,...
22950,39,2,1,중랑구
22951,39,2,2,중랑구
22952,19,2,1,중랑구
22953,20,2,1,중랑구


In [12]:
data_pbhlth_code = pd.read_excel('../raw/지역사회건강조사 2019년 원시자료 이용지침서 등/지역사회건강조사 2019 원시자료 참고사항_보건소정보, 세대유형.xlsx',header=3)
data_pbhlth_code = data_pbhlth_code[data_pbhlth_code['시도코드'] == 11][['보건소코드','보건소명']]
data_pbhlth_code["보건소명"] = data_pbhlth_code["보건소명"].str.replace('보건소', '', regex=True)
data_pbhlth_code.columns=['보건소코드','자치구']
list_region = list(data_pbhlth_code.to_dict()['자치구'].values()) + ['서울시']
list_region.reverse()

In [13]:
list_df = []
for value in list_region:
    temp = df_ne2['자치구'] == value
    list_df.append(df_ne2[temp])
list_df[0] = df_ne2
list_df

[       만나이  성별  사회환경_동네 자연환경  자치구
 0       82   2             1  강남구
 1       51   2             1  강남구
 2       26   1             2  강남구
 3       24   1             1  강남구
 4       27   2             1  강남구
 ...    ...  ..           ...  ...
 22950   39   2             1  중랑구
 22951   39   2             2  중랑구
 22952   19   2             1  중랑구
 22953   20   2             1  중랑구
 22954   27   1             2  중랑구
 
 [22700 rows x 4 columns],
        만나이  성별  사회환경_동네 자연환경   자치구
 17450   73   2             1  영등포구
 17451   25   1             2  영등포구
 17452   26   2             1  영등포구
 17453   85   2             2  영등포구
 17454   57   1             1  영등포구
 ...    ...  ..           ...   ...
 18366   71   2             2  영등포구
 18367   58   2             1  영등포구
 18368   85   2             2  영등포구
 18369   61   1             2  영등포구
 18370   26   2             1  영등포구
 
 [920 rows x 4 columns],
        만나이  성별  사회환경_동네 자연환경  자치구
 16531   72   2             1  양천구
 16532   68   2       

In [14]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

list_age = ['19~200','19~44','45~64','65~74','75~200']
list_sex = ['계','남','여']
idx_sex = []
idx_age = []
idx_region = []
value_ne = []
for i, value1 in enumerate(list_region):
    for j in list_age:
        for k in list_sex:            
            temp = list_df[i][list_df[i]['만나이']>=int(j.split('~')[0])]
            temp1 = temp[temp['만나이'] <= int(j.split('~')[1])]
            if k == '남':
                temp2 = temp1[temp1['성별'] == 1]
            elif k == '여':
                temp2 = temp1[temp1['성별'] == 2]
            else:
                temp2 = temp1
            value_ne.append(round(temp2['사회환경_동네 자연환경'].value_counts(normalize = True)[1]*100,1))
            if j == '19~200':
                idx_age.append('계')
            elif j == '75~200':
                idx_age.append('75 이상')
            else:
                idx_age.append(j)
            idx_sex.append(k)
            idx_region.append(value1)

In [15]:
df_final = pd.DataFrame({'긍정적 태도율(자연환경)':value_ne, '지역':idx_region,'연령별':idx_age,'성별':idx_sex})

df_final = df_final.set_index(['지역','연령별','성별'])

df_final

긍정적 태도율(자연환경)
지역  연령별   성별               
서울시 계     계            70.1
          남            71.7
          여            68.8
    19~44 계            64.4
          남            66.4
...                     ...
강남구 65~74 남            69.0
          여            77.1
    75 이상 계            93.8
          남           100.0
          여            87.5

[390 rows x 1 columns]

In [16]:
df_final.to_csv("../raw/지역사회건강조사 (전처리된 데이터)_2017/지역사회건강조사 (긍정적 태도율(자연환경))_2017.csv", mode='w',encoding = 'utf-8-sig')